In [2]:
def create_circle(m,r,ls='-'):
    return patches.Circle((m,0), radius=r, angle=180.,lw=2,fill=False,ls=ls)

In [3]:
def principal_to_pq(s_1fe,s_3fe):
    return (s_1fe + s_3fe)/2, (s_1fe - s_3fe)/2

In [4]:
def find_b_tanalpha(s_1fe,s_3fe):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    regr = sp.stats.linregress(s_m,t_m)
    return regr.slope, regr.intercept

In [5]:
def find_c_sinphi(s_1fe,s_3fe):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    regr = sp.stats.linregress(s_m,t_m)
    return regr.slope, regr.intercept/np.cos(np.arcsin(regr.slope))

In [6]:
def find_failure_stresses(s_1fe,s_3fe):
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    phi = np.arcsin(sinphi)
    theta = np.pi/4 + phi/2
    tau_alpha = (c + s_3fe*np.tan(phi))/(1-np.tan(phi)/np.tan(theta))
    s_alpha = (tau_alpha - c)/np.tan(phi)
    return s_alpha, tau_alpha

In [7]:
def plot_basic_circles(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.set(aspect='equal')
    ax.set_xlabel(r"$\sigma'_\mathrm{n}$ / kPa")
    ax.set_ylabel(r'$\tau$ / kPa')
    #ax.legend(fontsize=12)
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [8]:
def plot_pq(s_1fe,s_3fe,u=0):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    m, b = find_b_tanalpha(s_1fe,s_3fe)
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.set_xlabel(r"$\sigma_\mathrm{M}'$ / kPa")
    ax.set_ylabel(r"$(\sigma_1 - \sigma_3) / 2$ / kPa")
    ax.plot(s_m,t_m,ls='',marker='d',label=r"$\sigma'_\mathrm{M} = (\sigma_1'+\sigma_3')/2$")
    
    ax.plot([0,np.max(s_m)],[b,b+m*np.max(s_m)],label=r"$b' + \sigma_\mathrm{M}' \tan \alpha'$")
    fig.tight_layout()
    ax.legend()
    ax.set_xlim(0,np.max(s_m)*1.02)
    ax.set_ylim(0,np.max(t_m)*1.02)
    plt.show();

In [9]:
def plot_circles_tangent(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label=r"$c' + \sigma_\mathrm{n}' \tan \varphi'$")
    ax.set(aspect='equal')
    ax.set_xlabel(r"$\sigma'_\mathrm{n}$ / kPa")
    ax.set_ylabel(r'$\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [10]:
def plot_circles_failure(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    s_a, t_a = find_failure_stresses(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label=r"$c' + \sigma_\mathrm{n}' \tan \varphi'$")
    ax.plot(s_a,t_a,ls='',marker='o', label=r"$\sigma_\vartheta', \tau_\vartheta$",color='red')
    for i in range(len(s_1fe)):
        ax.plot([s_3fe[i],s_a[i]],[0,t_a[i]],ls='--',color='red')
    ax.set(aspect='equal')
    ax.set_xlabel(r"$\sigma'_\mathrm{n}$ / kPa")
    ax.set_ylabel(r'$\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [11]:
def plot_pq_tot(s_1fe,s_3fe,u=0,u0=0):
    s_me, t_me = principal_to_pq(s_1fe,s_3fe)
    m, b = find_b_tanalpha(s_1fe,s_3fe)
    
    s_conf = s_3fe+u
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.set_xlabel(r"$\sigma_\mathrm{M}$ / kPa")
    ax.set_ylabel(r"$(\sigma_1 - \sigma_3) / 2$ / kPa")
    ax.plot(s_me,t_me,ls='',marker='d',label=r"$\sigma_\mathrm{M} = (\sigma_1'+\sigma_3')/2$")
    ax.plot(s_me+u,t_me,ls='',marker='o',label=r"$\sigma_\mathrm{M} = (\sigma_1+\sigma_3)/2$")
    ax.plot(s_3fe+u,s_3fe*0,ls='',marker='s',label=r"$\sigma_\mathrm{M} = \sigma_\mathrm{c} + u_0 = \sigma_3$")
    
    ax.plot([0,np.max(s_me)],[b,b+m*np.max(s_me)],label=r"$b' + \sigma_\mathrm{M}' \tan \alpha'$")
    
    
    for i in range(len(s_1fe)):
        t = np.linspace(0,t_me[i],40)
        du = np.power(np.linspace(0,u[i]-u0[i],40)/(u[i]-u0[i]+1.e-14),3)*(u[i]-u0[i]) + u0[i]
        ds = np.linspace(s_3fe[i]+u[i],s_me[i]+u[i],40)
        if (i==0):
            ax.plot([0,s_3fe[i]+u[i],s_me[i]+u[i]],[0,0,t_me[i]],color='green',label=r'Totalspannungspfad')
            ax.plot(ds-du,t,color='red',label=r'Effektivpannungspfad',ls='--')
        else:
            ax.plot([0,s_3fe[i]+u[i],s_me[i]+u[i]],[0,0,t_me[i]],color='green')
            ax.plot(ds-du,t,color='red',ls='--')
        for j in range(len(t)):
            if (np.abs(du[j])>10):
                ax.annotate(r"", xy=(ds[j]-du[j], t[j]), xytext=(ds[j], t[j]), arrowprops=dict(arrowstyle="->",color='blue'))
            else:
                ax.annotate(r"", xy=(ds[j]-du[j], t[j]), xytext=(ds[j], t[j]), arrowprops=dict(arrowstyle="-",color='blue'))
    
    fig.tight_layout()
    ax.set_xlim(0,np.max(s_me+u)*1.02)
    ax.set_ylim(0,np.max(t_me)*1.02)
    ax.legend()
    plt.show();

In [1]:
def plot_circles_tangent_total(s_1fe,s_3fe,uf):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))
    
    s_mt, t_mt = principal_to_pq(s_1fe+uf,s_3fe+uf)
    sinphiu, cu = find_c_sinphi(s_1fe+uf,s_3fe+uf)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_mt[i],t_mt[i],':'))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label=r"$c' + \sigma_\mathrm{n}' \tan \varphi'$")
    ax.plot([0,np.max(s_1fe+uf)],[cu,cu+np.max(s_1fe+uf)*np.tan(np.arcsin(sinphiu))],label=r"$c_\mathrm{u} + \sigma_\mathrm{n} \tan \varphi_\mathrm{u}$")
    ax.set(aspect='equal')
    ax.set_xlabel(r"$\sigma'_\mathrm{n}$, $\sigma_\mathrm{n}$ / kPa")
    ax.set_ylabel(r'$\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe+uf))
    ax.set_ylim(0,np.max(s_1fe+uf)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [2]:
def plot_circles_failure_1():
    fig, ax = plt.subplots(figsize=(12,12))

    #Definition Bruchspannung
    phi = np.deg2rad(30.)
    s_nf = 500.
    tau_f = s_nf*np.tan(phi)

    #Bruchspannung
    ax.plot(s_nf,tau_f,marker='d',color='red',markersize='8',ls='',label=r'$(\sigma^*,\tau^*)$')

    theta = np.pi/4+phi/2
    s3_1 = s_nf - tau_f/np.tan(theta)
    s1_1 = (1.+np.sin(phi))/(1.-np.sin(phi))*s3_1
    sm_1 = (s3_1 + s1_1)/2
    tm_1 = (s1_1 - s3_1)/2
    ax.add_patch(create_circle(sm_1,tm_1))
    #Schergerade
    ax.plot([0,s1_1],[0,s1_1*np.tan(phi)],label=r"$\sigma_\mathrm{n}' \tan \varphi'$",color='green')
    #Pol
    sp_1 = s1_1- tau_f/np.tan(theta)
    ax.plot(sp_1,tau_f,marker='o',color='green')
    ax.annotate(r'$P$',xy=(sp_1,tau_f),color='green')
    #Hauptspannungen
    ax.plot([s3_1,s1_1],[0,0],marker='s',ls='',color='green')
    ax.plot([s3_1,sp_1,s1_1],[0,tau_f,0],ls='--',color='green')

    ax.set(aspect='equal')
    ax.set_xlabel(r"$\sigma'_\mathrm{n}$ / kPa")
    ax.set_ylabel(r'$\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,s1_1*1.01)
    ax.set_ylim(bottom=0)
    ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [ ]:
def plot_circles_failure_2():
    fig, ax = plt.subplots(figsize=(12,12))

    #Definition Bruchspannung
    phi = np.deg2rad(30.)
    s_nft = 500.
    tau_ft = s_nft*np.tan(phi)


    theta = np.pi/4+phi/2
    s3_1 = s_nft - tau_ft/np.tan(theta)
    s1_1 = (1.+np.sin(phi))/(1.-np.sin(phi))*s3_1
    sm_1 = (s3_1 + s1_1)/2
    tm_1 = (s1_1 - s3_1)/2
    
    #Bruchspannung
    s_nf = sm_1
    tau_f = tm_1
    ax.plot(s_nf,tau_f,marker='d',color='red',markersize='8',ls='',label=r'$(\sigma^*,\tau^*)$')
    
    ax.add_patch(create_circle(sm_1,tm_1))
    #Schergerade
    ax.plot([0,s1_1],[0,s1_1*np.tan(phi)],label=r"$\sigma_\mathrm{n}' \tan \varphi'$",color='green',ls=':')

    s3_2 = s_nf - tau_f
    s1_2 = s_nf + tau_f
    ax.add_patch(create_circle(s_nf,tau_f))
    #Schergerade
    ax.plot([0,s1_2],[0,s1_2*tau_f/s_nf],label=r"$\sigma_\mathrm{n}' \tan \phi'$",color='red')
    #Pol
    ax.annotate(r'$P$',xy=(s_nf,tau_f),color='red')
    #Hauptspannungen
    ax.plot([s3_2,s1_2],[0,0],marker='s',ls='',color='red')
    ax.plot([s3_2,s_nf,s1_2],[0,tau_f,0],ls='--',color='red')   

    ax.set(aspect='equal')
    ax.set_xlabel(r"$\sigma'_\mathrm{n}$ / kPa")
    ax.set_ylabel(r'$\tau$ / kPa')
    ax.set_xlim(0,s1_1*1.01)
    ax.set_ylim(bottom=0)
    ax.spines['left'].set_position(('data', 0))
    ax.legend()
    fig.tight_layout()
    plt.show();

In [ ]:
def plot_variants():
    #s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    #m, b = find_b_tanalpha(s_1fe,s_3fe)
    c = 30
    phi = np.deg2rad(30)
    b = c * np.cos(phi)
    m = np.sin(phi)
    
    s_m = np.linspace(0,800,2)
    #
    s_ax_comp = lambda phi, c, sig3: ((1+np.sin(phi)) * sig3 + 2*c*np.cos(phi))/(1-np.sin(phi))
    p_conf_comp = lambda phi, c, sig1: (sig1*(1-np.sin(phi))-2*c*np.cos(phi))/(1+np.sin(phi))
    s_ax_ext = lambda phi, c, sig1: ((1-np.sin(phi)) * sig1 - 2*c*np.cos(phi))/(1+np.sin(phi))
    p_conf_ext = lambda phi, c, sig3: (sig3*(1+np.sin(phi))+2*c*np.cos(phi))/(1-np.sin(phi))
    
    #ISO
    s0 = 400.
    
    #pS, aS, aD, pD
    sh = [s0,p_conf_comp(phi,c,s0),s0,p_conf_ext(phi,c,s0)]
    sv = [s_ax_comp(phi,c,s0),s0,s_ax_ext(phi,c,s0),s0]
    slabels = ['pS','aS','aD','pD']
    
    fig, ax = plt.subplots(figsize=(18,8),ncols=2)
    ax[0].set_xlabel(r"$\sigma_\mathrm{M}$ / kPa")
    ax[0].set_ylabel(r"$(\sigma_1 - \sigma_3) / 2$ / kPa")
    #ax[0].plot(s_m,t_m,ls='',marker='d',label=r"$\sigma_\mathrm{M} = (\sigma_1'+\sigma_3')/2$")
    
    ax[0].plot(s_m*1.2,b+m*s_m*1.2,label=r"Coulomb",ls='--',color='black')
    ax[0].plot(s_m*1.2,-b-m*s_m*1.2, ls='--',color='black')
    #
    ax[0].plot(s0,0,marker='s',ls='',color='black',label=r'isotrop konsolidiert')
    for i in range(len(sh)):
        ax[0].plot([s0,(sh[i]+sv[i])/2],[0,(sv[i]-sh[i])/2],label=slabels[i])
    
    ax[1].plot(s_m*0.8,s_ax_comp(phi,c,s_m*0.8),ls='--',label=r'Coulomb',color='black')
    ax[1].plot(s_m*2,s_ax_ext(phi,c,s_m*2),ls='--',color='black')
    #
    ax[1].set_ylabel(r'$\sigma_\mathrm{axial}$ / kPa')
    ax[1].set_xlabel(r'$p_\mathrm{Zell}$ / kPa')
    for i in range(len(sh)):
        ax[1].plot([s0,sh[i]],[s0,sv[i]],label=slabels[i])
    #
    ax[1].plot(s_m,s_m,ls='--',label=r'$\sigma_1 = \sigma_2 = \sigma_3$')
    ax[1].plot(s0,s0,marker='s',ls='',color='black',label=r'isotrop konsolidiert')
    
    ax[1].legend()
    
    for i in range(2):
        ax[i].spines['bottom'].set_position(('data', 0))
        ax[i].set_xlim(left=0)
   
    fig.tight_layout();